In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback
from sklearn.metrics import recall_score, precision_score, f1_score
from keras.applications import DenseNet121
from keras.applications.resnet import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalMaxPooling2D, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
Train_data = np.load('../input/iwildcam-2019-inat-idaho-32x32/X_train.npy')
Test_data = np.load('../input/iwildcam-2019-inat-idaho-32x32/X_test.npy')
Train_label = np.load('../input/iwildcam-2019-inat-idaho-32x32/y_train.npy')

In [ ]:
#shift = 0.2
#datagen = ImageDataGenerator(zca_whitening = True, rotation_range = 90, width_shift_range = shift, height_shift_range = range,
                          #   brightness_range = [0.2,1], channel_shift_range = 200, horizontal_flip = True,
                           #  vertical_flip = True)
#datagen.fit(Train_data)

In [ ]:
print(Train_data.shape)

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs = {}):
        self.val_f1 = []
        self.val_precision = []
        self.val_recall = []
        
    def on_epoch_end(self, epoch, logs ={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_target = self.validation_data[1]
        _val_f1 = f1_score(val_target, val_predict, average = 'macro')
        _val_precision = precision_score(val_target, val_predict, average = 'macro')
        _val_recall = recall_score(val_target, val_predict, average = 'macro')
        self.val_f1.append(_val_f1)
        self.val_precision.append(_val_precision)
        self.val_recall.append(_val_recall)
        print("Val_f1 = %f -- Val_precision = %f -- Val_recall = %f", _val_f1, _val_precision, _val_recall)
        return

In [ ]:
num_classes = 23
epochs = 50
val_split = 0.2
batch_size = 64

In [ ]:
#densenet = DenseNet121(weights=None, include_top = False, input_shape = (32, 32, 3))

In [ ]:
model = Sequential()
#model.add(densenet)
#model.add(resnet)
#model.add(GlobalMaxPooling2D())
model.add(Conv2D(32, (3,3), padding = 'same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding = 'same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding = 'same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#model.add(Conv2D(256, (3,3), padding = 'same'))
#model.add(Activation('relu'))
#model.add(Conv2D(256, (3,3), padding = 'same'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(1024))
#model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
#model.layers[0].trainable = False

In [ ]:
f1_metrics = Metrics()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
hist = model.fit(Train_data, Train_label, batch_size = batch_size, epochs = epochs, validation_split=val_split, 
                callbacks = [f1_metrics], shuffle = True)

In [ ]:
model.summary()

In [ ]:
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'keras_CNN_model.h5'

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
#model.save(model_path)

print("Saved the model at %s", model_path)

In [ ]:
#model.load_weights('keras_CNN_model.h5')
y_test = model.predict(Test_data)
submission_df = pd.read_csv('../input/iwildcam-2019/submission.csv')
submission_df['Predicted'] = y_test.argmax(axis=1)

print(submission_df.shape)

submission_df.to_csv('Submission.csv', index = False)